# Downsample the WT Gold data to 30 fps

In [ ]:
%reset -f
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sys

from disp import set_plot

NTR = 84
FPS_UP = 150
FPS_DN = 30.03
DT_DN = 1/FPS_DN

df_up = pd.read_csv('data/simple/w_song_f_behav_up.csv')
df_up

In [ ]:
dfs_tr_up = [df_up[df_up['ID'] == ctr] for ctr in range(NTR)]

In [ ]:
# loop over trials and downsample via moving average
dfs_tr_dn = []
for ctr, df_tr_up in enumerate(dfs_tr_up):
    sys.stdout.write('.')
    t_up = np.array(df_tr_up['T'])
    t_max = t_up[-1]
    
    t = np.arange(0, t_max, DT_DN)
    q = np.zeros(len(t), dtype=int)
    s = np.zeros(len(t), dtype=int)
    p = np.zeros(len(t), dtype=int)
    f = np.zeros(len(t), dtype=int)
    
    ffv = np.nan * np.zeros(len(t))
    ffa = np.nan * np.zeros(len(t))
    fls = np.nan * np.zeros(len(t))
    frs = np.nan * np.zeros(len(t))
    fmfv = np.nan * np.zeros(len(t))
    fmls = np.nan * np.zeros(len(t))
    
    for ct, t_start in enumerate(t):
        t_end = t_start + DT_DN
        mt = (t_start <= t_up) & (t_up < t_end)
        
        # get which song mode dominates in this time window
        ct_q = np.sum(df_tr_up['Q'][mt])
        ct_s = np.sum(df_tr_up['S'][mt])
        ct_p = np.sum(df_tr_up['P'][mt])
        ct_f = np.sum(df_tr_up['F'][mt])
        
        if ct_s == 0 and ct_p == 0 and ct_f == 0:
            # only call time point quiet if no sine or pulse at all
            imode = 0
        else:
            # otherwise find dominant mode
            imode = np.argmax([-np.inf, ct_s, ct_p, ct_f])
            
        if imode == 0:
            q[ct] = 1
        elif imode == 1:
            s[ct] = 1
        elif imode == 2:
            p[ct] = 1
        elif imode == 3:
            f[ct] = 1
            
        # take mean of fbehav vars
        ffv[ct] = np.array(df_tr_up['FFV'])[mt].mean()
        ffa[ct] = np.array(df_tr_up['FFA'])[mt].mean()
        fls[ct] = np.array(df_tr_up['FLS'])[mt].mean()
        frs[ct] = np.array(df_tr_up['FRS'])[mt].mean()
        fmfv[ct] = np.array(df_tr_up['FMFV'])[mt].mean()
        fmls[ct] = np.array(df_tr_up['FMLS'])[mt].mean()
        
    df_tr_dn = pd.DataFrame(
        columns=['ID', 'FRAME', 'T', 'Q', 'S', 'P', 'F', 'FFV', 'FFA', 'FLS', 'FRS'],
        data={
            'ID': np.repeat(ctr, len(t)),
            'FRAME': np.arange(len(t)),
            'T': t,
            'Q': q,
            'S': s,
            'P': p,
            'F': f,
            'FFV': ffv,
            'FFA': ffa,
            'FLS': fls,
            'FRS': frs,
            'FMFV': fmfv,
            'FMLS': fmls,
        })

    dfs_tr_dn.append(df_tr_dn)

In [ ]:
# concatenate and save dataframes
df_dn = pd.concat(dfs_tr_dn)
df_dn.to_csv('data/simple/w_song_f_behav.csv', index=False)

In [ ]:
# load down sampled data and plot atop original
df_dn_reloaded = pd.read_csv('data/simple/w_song_f_behav.csv')
dfs_tr_dn_reloaded = [df_dn_reloaded[df_dn_reloaded['ID'] == ctr] for ctr in range(NTR)]
df_dn_reloaded

In [ ]:
for ctr in range(6):
    fig, axs = plt.subplots(4, 1, figsize=(14, 9), tight_layout=True, sharex=True)
    
    t_up = dfs_tr_up[ctr]['T']
    q_up = dfs_tr_up[ctr]['Q']
    s_up = dfs_tr_up[ctr]['S']
    p_up = dfs_tr_up[ctr]['P']
    f_up = dfs_tr_up[ctr]['F']
    
    ffv_up = dfs_tr_up[ctr]['FFV']
    fls_up = dfs_tr_up[ctr]['FLS']
    
    t_dn = dfs_tr_dn_reloaded[ctr]['T']
    q_dn = dfs_tr_dn_reloaded[ctr]['Q']
    s_dn = dfs_tr_dn_reloaded[ctr]['S']
    p_dn = dfs_tr_dn_reloaded[ctr]['P']
    f_dn = dfs_tr_dn_reloaded[ctr]['F']
    
    ffv_dn = dfs_tr_dn_reloaded[ctr]['FFV']
    fls_dn = dfs_tr_dn_reloaded[ctr]['FLS']
    
    axs[0].plot(t_up, q_up, color='gray')
    axs[0].plot(t_up, s_up+1.2, color='b')
    axs[0].plot(t_up, p_up+2.4, color='r')
    axs[0].plot(t_up, f_up+3.6, color='orange')
    axs[0].set_title(f'Original song (TRIAL {ctr})')
    
    axs[1].plot(t_dn, q_dn, color='gray')
    axs[1].plot(t_dn, s_dn+1.2, color='b')
    axs[1].plot(t_dn, p_dn+2.4, color='r')
    axs[1].plot(t_dn, f_dn+3.6, color='orange')
    axs[1].set_title(f'Downsampled song (TRIAL {ctr})')
    
    axs[2].plot(t_up, ffv_up, c='k')
    axs[2].plot(t_dn, ffv_dn, c='m')
    axs[2].set_title(f'FFV (TRIAL {ctr})')
    axs[2].legend(['Original', 'Downsampled'])
    
    axs[3].plot(t_up, fls_up, c='k')
    axs[3].plot(t_dn, fls_dn, c='m')
    axs[3].set_title(f'FLS (TRIAL {ctr})')
    axs[3].legend(['Original', 'Downsampled'])
    
    for ax in axs:
        set_plot(ax, x_label='Time (s)', font_size=16)

Zoom in on specific song window

In [ ]:
ctr = 0
fig, axs = plt.subplots(4, 1, figsize=(14, 9), tight_layout=True)

t_up = dfs_tr_up[ctr]['T']
q_up = dfs_tr_up[ctr]['Q']
s_up = dfs_tr_up[ctr]['S']
p_up = dfs_tr_up[ctr]['P']
f_up = dfs_tr_up[ctr]['F']

ffv_up = dfs_tr_up[ctr]['FFV']
fls_up = dfs_tr_up[ctr]['FLS']

t_dn = dfs_tr_dn_reloaded[ctr]['T']
q_dn = dfs_tr_dn_reloaded[ctr]['Q']
s_dn = dfs_tr_dn_reloaded[ctr]['S']
p_dn = dfs_tr_dn_reloaded[ctr]['P']
f_dn = dfs_tr_dn_reloaded[ctr]['F']

ffv_dn = dfs_tr_dn_reloaded[ctr]['FFV']
fls_dn = dfs_tr_dn_reloaded[ctr]['FLS']

axs[0].plot(t_up*6, q_up, color='gray')
axs[0].plot(t_up*6, s_up+1.2, color='b')
axs[0].plot(t_up*6, p_up+2.4, color='r')
axs[0].plot(t_up*6, f_up+3.6, color='orange')
axs[0].set_title(f'Original song (TRIAL {ctr})')

axs[1].plot(t_dn*6, q_dn, color='gray')
axs[1].plot(t_dn*6, s_dn+1.2, color='b')
axs[1].plot(t_dn*6, p_dn+2.4, color='r')
axs[1].plot(t_dn*6, f_dn+3.6, color='orange')
axs[1].set_title(f'Downsampled song (TRIAL {ctr})')

axs[2].plot(t_up*6, ffv_up, c='k')
axs[2].plot(t_dn*6, ffv_dn, c='m')
axs[2].set_title(f'FFV (TRIAL {ctr})')
axs[2].legend(['Original', 'Downsampled'])

axs[3].plot(t_up*6, fls_up, c='k')
axs[3].plot(t_dn*6, fls_dn, c='m')
axs[3].set_title(f'FLS (TRIAL {ctr})')
axs[3].legend(['Original', 'Downsampled'])

for ax in axs:
    ax.set_xlim(1200,1500)
    set_plot(ax, x_label='Video time (s) (slowed down 6x)', font_size=16)